In [2]:
import json

import pandas as pd
import numpy as np

In [3]:
df_raw = pd.read_csv('data/goemotions.csv')

In [4]:
emotions_cols = ['admiration',
       'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 'neutral']

In [5]:
emotion_categories = {'admiration': 'Positive', # Proposed simplified classification
                      'amusement':'Positive',
                      'anger':'Negative',
                      'annoyance':'Negative',
                      'approval':'Positive',
                      'caring':'Positive',
                      'confusion':'Neutral',
                      'curiosity':'Positive',
                      'desire':'Positive',
                      'disappointment':'Negative',
                      'disapproval':'Negative',
                      'disgust':'Negative',
                      'embarrassment':'Neutral',
                      'excitement':'Positive',
                      'fear':'Negative',
                      'gratitude':'Positive',
                      'grief':'Negative',
                      'joy':'Positive',
                      'love':'Positive',
                      'nervousness':'Neutral',
                      'optimism':'Positive',
                      'pride':'Positive',
                      'realization':'Neutral',
                      'relief':'Positive',
                      'remorse':'Negative',
                      'sadness':'Negative',
                      'surprise':'Neutral',
                      'neutral':'Neutral'
                       }

In [6]:
emotion_groups = {'admiration': 1, 
                      'amusement':2,
                      'anger':3,
                      'annoyance':4,
                      'approval':5,
                      'caring':6,
                      'confusion':7,
                      'curiosity':8,
                      'desire':9,
                      'disappointment':10,
                      'disapproval':11,
                      'disgust':12,
                      'embarrassment':13,
                      'excitement':14,
                      'fear':15,
                      'gratitude':16,
                      'grief':17,
                      'joy':18,
                      'love':19,
                      'nervousness':20,
                      'optimism':21,
                      'pride':22,
                      'realization':23,
                      'relief':24,
                      'remorse':25,
                      'sadness':26,
                      'surprise':27,
                      'neutral':28
                       }

In [7]:
df = df_raw.set_index(['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id',
       'created_utc', 'rater_id', 'example_very_unclear'])

df = df[df==1].stack().reset_index().drop(0,1).rename(columns = {'level_9': 'sentiment'})
df.head()

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,sentiment
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1548381039,1,False,sadness
1,"You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,1546427744,37,False,neutral
2,Man I love reddit.,eeibobj,MrsRobertshaw,facepalm,t3_ahulml,t3_ahulml,1547965054,18,False,love
3,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,American_Fascist713,starwarsspeculation,t3_ackt2f,t1_eda65q2,1546668601,2,False,neutral
4,Right? Considering it’s such an important docu...,eespn2i,ImperialBoss,TrueReddit,t3_aizyuz,t1_eesoak0,1548280208,61,False,gratitude


In [8]:
df['sentiment_rating'] = df['sentiment'].map(emotion_categories).map({ 'Negative': -1, "Neutral": 0, "Positive": 1})

df.head()

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,sentiment,sentiment_rating
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1548381039,1,False,sadness,-1
1,"You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,1546427744,37,False,neutral,0
2,Man I love reddit.,eeibobj,MrsRobertshaw,facepalm,t3_ahulml,t3_ahulml,1547965054,18,False,love,1
3,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,American_Fascist713,starwarsspeculation,t3_ackt2f,t1_eda65q2,1546668601,2,False,neutral,0
4,Right? Considering it’s such an important docu...,eespn2i,ImperialBoss,TrueReddit,t3_aizyuz,t1_eesoak0,1548280208,61,False,gratitude,1


In [9]:
# Get top sentiment for each subreddit
top_sentiment_per_subreddit = df[df['sentiment'] != 'neutral'].groupby(['subreddit', 'sentiment'], as_index = False).count().sort_values(['subreddit', 'text'], ascending = False).groupby('subreddit', as_index = False).first()[['subreddit', 'sentiment', 'text']].rename(columns = { 'text': 'n', 'subreddit': 'name'})

top_sentiment_per_subreddit['group'] = top_sentiment_per_subreddit['sentiment'].map(emotion_groups)
top_sentiment_per_subreddit['id'] = top_sentiment_per_subreddit['name']

top_sentiment_per_subreddit.head()

,name,sentiment,n,group,id
0,2meirl4meirl,sadness,70,26,2meirl4meirl
1,49ers,admiration,50,1,49ers
2,4PanelCringe,amusement,73,2,4PanelCringe
3,90DayFiance,admiration,76,1,90DayFiance
4,90dayfianceuncensored,approval,63,5,90dayfianceuncensored


In [10]:
nodes = top_sentiment_per_subreddit.to_dict('records')

In [31]:
top_sentiment_per_subreddit[top_sentiment_per_subreddit['name'] == 'doordash']

,name,sentiment,n,group,id
298,doordash,approval,45,5,doordash


In [22]:
df[df['author'] == '[deleted]']

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,sentiment,sentiment_rating
9,"I have, and now that you mention it, I think t...",ed9w1hm,[deleted],AskMenOver30,t3_acm7f6,t1_ed9m2l0,1546658263,36,False,neutral,0
14,Build a wall? /jk,edsqvyx,[deleted],AskMenOver30,t3_aeq3ep,t1_edsn1bd,1547207519,10,False,neutral,0
108,I didn't like the family ending but I liked th...,ee6mb6z,[deleted],assassinscreed,t3_agboi2,t1_ee6m488,1547628831,56,False,approval,1
124,My fans on patreon will be rewarded soon,ef8s6ku,[deleted],nrl,t3_akwycz,t1_ef8pvmo,1548746680,33,False,neutral,0
181,"Integrated, Antwaarps speaking, second generat...",eepltiz,[deleted],belgium,t3_ai8u8j,t1_eem1uan,1548184008,34,False,disgust,-1
...,...,...,...,...,...,...,...,...,...,...,...
249068,This post is inappropriate for this subreddit.,edau7nd,[deleted],SubredditDrama,t3_acuh2o,t3_acuh2o,1546698714,39,False,disapproval,-1
249145,This post is inappropriate for this subreddit.,edau7nd,[deleted],SubredditDrama,t3_acuh2o,t3_acuh2o,1546698714,26,False,disapproval,-1
249164,This post is inappropriate for this subreddit.,edau7nd,[deleted],SubredditDrama,t3_acuh2o,t3_acuh2o,1546698714,37,False,realization,0
249199,Fcking disgusting.,edwux5m,[deleted],COMPLETEANARCHY,t3_af8jgr,t3_af8jgr,1547319348,35,False,disgust,-1


In [32]:
# graph links
links_data = []
df_clean = df[df['author'] != '[deleted]']
for index, row in df_clean.iterrows():
    author = row['author']
    other_posts = df_clean[(df_clean['id'] != row['id']) & (df_clean['author'] == author)]
    other_subreddits = list(set(other_posts['subreddit']))
    for subreddit in other_subreddits:
        links_data.append({ 'source': row['subreddit'], 'target': subreddit, 'count': 1, 'author': author})
    
    clear_output()
    print(index, "/", len(df_clean))


249528 / 244928


In [28]:
test = pd.DataFrame(links_data)
test[test['source'] == 'Advice']

,source,target,count,author
627,Advice,nononono,1,Daymandayman
872,Advice,TheSimpsons,1,createusername32
873,Advice,TheSimpsons,1,createusername32
1028,Advice,Advice,1,theshinyspacelord
1685,Advice,korea,1,Steffy_love
2396,Advice,doordash,1,BlowingSmokeUpYourAs
2850,Advice,nononono,1,Daymandayman
3041,Advice,DoesAnybodyElse,1,haramishirin
3042,Advice,DoesAnybodyElse,1,haramishirin
3043,Advice,DoesAnybodyElse,1,haramishirin


In [11]:
from IPython.display import clear_output

In [39]:
links_df[links_df['source'] == 'atheism']

,source,target,value
3169,atheism,DebateAnAtheist,1
3184,atheism,atheism,1
3188,atheism,exchristian,1
3193,atheism,loveafterlockup,1
3164,atheism,ATBGE,1
3165,atheism,Anarchism,1
3166,atheism,BetterEveryLoop,1
3167,atheism,ChoosingBeggars,1
3168,atheism,Christianity,1
3170,atheism,Denver,1


In [38]:
links_df = pd.DataFrame(links_data).drop_duplicates(subset = ['source', 'target', 'author'], keep = "first").groupby(['source', 'target'], as_index = False).count().sort_values(['source', 'count'], ascending = False).groupby(['source'], as_index = False).head(20)[['source', 'target']]

links_df['value'] = 1

links_df.head()

,source,target,value
6159,youtubehaiku,DiWHY,1
6160,youtubehaiku,KotakuInAction,1
6161,youtubehaiku,SubredditDrama,1
6162,youtubehaiku,WhitePeopleTwitter,1
6163,youtubehaiku,antiMLM,1


In [40]:
links = links_df.to_dict('records')

In [41]:
arc_diagram_graph = {
    'nodes': nodes,
    'links': links
}

with open('arc-data.json', 'w') as fout:
    json.dump(arc_diagram_graph , fout)

In [62]:
# For Network graph
df_users = df[['author', 'rater_id', 'subreddit', 'sentiment_rating']]
user_nodes = df_users.groupby(['author'], as_index = False).agg({ 'sentiment_rating': 'mean', 'subreddit': 'nunique'}).rename(columns = {'subreddit': 'connections', 'author': 'name'})
user_nodes['type'] = 'user'
user_nodes = user_nodes.to_dict('records')

subreddit_nodes = df_users.groupby(['subreddit'], as_index = False).agg({ 'sentiment_rating': 'mean', 'author': 'nunique'}).rename(columns = {'author': 'connections', 'subreddit': 'name'})
subreddit_nodes['type'] = 'subreddit'
subreddit_nodes = subreddit_nodes.to_dict('records')

with open('nodes.json', 'w') as fout:
    json.dump(user_nodes + subreddit_nodes , fout)

In [68]:
authors_and_subreddits = df_users[['author', "subreddit"]].drop_duplicates(keep = "first")
subreddits = authors_and_subreddits['subreddit'].drop_duplicates(keep = "first")

links = []

for subreddit in subreddits:
    authors = authors_and_subreddits[authors_and_subreddits['subreddit'] == subreddit]['author'].to_list()
    for author in authors:
        links.append({ 'source': subreddit, 'target': author})

with open('links.json', 'w') as fout:
    json.dump(links , fout)

In [67]:
len(links)

51833

In [56]:
len(authors)

49150

In [14]:
df = df_raw.groupby(['id', 'subreddit', 'text'])[emotions_cols].sum().reset_index() # Aggregates by post

In [6]:
# Only keeps emotion(s) with the most votes for each post
"""
df['max_val'] = df[emotions_cols].max(axis=1)
for col in emotions_cols:
    df[col] = df[col] - df['max_val']
del df['max_val']
df_new = (df.melt(['id', 'subreddit'], var_name='emotion').query('value >= 0')
       .groupby(['id', 'subreddit'])['emotion']
       .apply(', '.join)
       .reset_index())
df_grouped = df_new.groupby(['subreddit', 'emotion']).count().reset_index()
"""

"\ndf['max_val'] = df[emotions_cols].max(axis=1)\nfor col in emotions_cols:\n    df[col] = df[col] - df['max_val']\ndel df['max_val']\ndf_new = (df.melt(['id', 'subreddit'], var_name='emotion').query('value >= 0')\n       .groupby(['id', 'subreddit'])['emotion']\n       .apply(', '.join)\n       .reset_index())\ndf_grouped = df_new.groupby(['subreddit', 'emotion']).count().reset_index()\n"

In [15]:
df.groupby(['subreddit']).count().sort_values('id')

,id,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
subreddit,,,,,,,,,,,,,,,,,,,,,
farcry,25,25,25,25,25,25,25,25,25,25,...,25,25,25,25,25,25,25,25,25,25
shieldbro,32,32,32,32,32,32,32,32,32,32,...,32,32,32,32,32,32,32,32,32,32
Anticonsumption,34,34,34,34,34,34,34,34,34,34,...,34,34,34,34,34,34,34,34,34,34
SSBM,35,35,35,35,35,35,35,35,35,35,...,35,35,35,35,35,35,35,35,35,35
darknet,38,38,38,38,38,38,38,38,38,38,...,38,38,38,38,38,38,38,38,38,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vanderpumprules,221,221,221,221,221,221,221,221,221,221,...,221,221,221,221,221,221,221,221,221,221
AnimalsBeingBros,231,231,231,231,231,231,231,231,231,231,...,231,231,231,231,231,231,231,231,231,231
socialanxiety,232,232,232,232,232,232,232,232,232,232,...,232,232,232,232,232,232,232,232,232,232


In [20]:
subreddit_df = pd.DataFrame(columns = ['subreddit', 'emotion', 'count'])
for emotion in emotions_cols:
    gp_df = df.groupby(['subreddit'])[emotion].sum()
    d = {'subreddit': gp_df.index, 'emotion': [emotion]*len(gp_df), 'count':gp_df.values}
    subreddit_df = subreddit_df.append(pd.DataFrame(data = d), ignore_index=True)

In [9]:
#hv.extension('bokeh')
#hv.Sankey(subreddit_df.loc[lambda f: f['subreddit'] == 'socialanxiety'])

In [59]:
#subreddit_df.to_csv('subreddit_emotion.csv')

In [39]:
text_df = pd.DataFrame(columns = ['subreddit', 'emotion', 'text'])
for emotion in emotions_cols:
    df_long = df[df['text'].str.split().str.len().ge(4)] # Remove short posts
    df_sorted = df_long.sort_values(emotion, ascending=False)
    gp_df = df_sorted.groupby('subreddit').first()
    d = {'subreddit': gp_df.index, 'emotion': [emotion]*len(gp_df), 'text':gp_df['text']}
    text_df = text_df.append(pd.DataFrame(data = d), ignore_index=True)

In [44]:
full_df = pd.merge(text_df, subreddit_df, how='left', on=['subreddit', 'emotion'])

In [46]:
full_df.columns = ['source', 'target', 'text', 'value']

In [47]:
full_df.to_csv('subreddit_emotion_2.csv')

In [45]:
full_df

,subreddit,emotion,text,count
0,2meirl4meirl,admiration,Or embalmed! Green burial is the best burial,47
1,49ers,admiration,Can we please just call out defense next year ...,50
2,4PanelCringe,admiration,Seriously. [NAME] is a hero. I heard he was a ...,52
3,90DayFiance,admiration,maybe. Id praise her for it though. In a world...,76
4,90dayfianceuncensored,admiration,She could and should pursue modeling! It seeme...,43
...,...,...,...,...
13519,worldpolitics,neutral,The original video was debunked. Do you agree ...,156
13520,yesyesyesyesno,neutral,This is like 4 years old from vine,198
13521,youseeingthisshit,neutral,Imagine being upset over who someone loves.,198
13522,youtube,neutral,Naming comes from the heart,95
